In [1]:
import os
import json
from PIL import Image
folder_path = r'C:\Users\Ahmed\Desktop\dataset'  # Replace with the actual folder path

items = os.listdir(folder_path)
images_dict = {}
for item in items:
    item_path = os.path.join(folder_path, item)
    if os.path.isdir(item_path):
        iii = os.listdir(item_path)
        for iiii in iii:
                iiii_path = os.path.join(item_path, iiii)
                if iiii_path.endswith(".json"):
                # Read the json file to get the interest
                    with open(iiii_path) as json_file:
                        data = json.load(json_file)
                        interest = data["interests"]
                elif iiii_path.endswith("train"):
                    for file_name in os.listdir(iiii_path):
                        file_path = os.path.join(iiii_path, file_name)
                        # Add the image and its corresponding interest to the dictionary
                        images_dict[file_path] = interest
                    
print(images_dict)

{'C:\\Users\\Ahmed\\Desktop\\dataset\\1009078\\train\\BveCDRLlmr0.jpg': ['BEAUTY'], 'C:\\Users\\Ahmed\\Desktop\\dataset\\1009078\\train\\BvXtjZYFOHZ.jpg': ['BEAUTY'], 'C:\\Users\\Ahmed\\Desktop\\dataset\\1009078\\train\\By5AEQ1l02Q.jpg': ['BEAUTY'], 'C:\\Users\\Ahmed\\Desktop\\dataset\\1009078\\train\\By5AMGmlaCZ.jpg': ['BEAUTY'], 'C:\\Users\\Ahmed\\Desktop\\dataset\\1009078\\train\\BybslSDF-4N.jpg': ['BEAUTY'], 'C:\\Users\\Ahmed\\Desktop\\dataset\\1009078\\train\\ByFX2CPFi4J.jpg': ['BEAUTY'], 'C:\\Users\\Ahmed\\Desktop\\dataset\\1009078\\train\\ByH8Gk7lefd.jpg': ['BEAUTY'], 'C:\\Users\\Ahmed\\Desktop\\dataset\\1009078\\train\\ByJEhqfFUp4.jpg': ['BEAUTY'], 'C:\\Users\\Ahmed\\Desktop\\dataset\\1009078\\train\\BylQrOZlNcY.jpg': ['BEAUTY'], 'C:\\Users\\Ahmed\\Desktop\\dataset\\1009078\\train\\BylT9eflZIC.jpg': ['BEAUTY'], 'C:\\Users\\Ahmed\\Desktop\\dataset\\1009078\\train\\BylTarslyHz.jpg': ['BEAUTY'], 'C:\\Users\\Ahmed\\Desktop\\dataset\\1009078\\train\\BylUWATlR4H.jpg': ['BEAUTY'], 'C:

In [2]:
from PIL import Image

# Assuming you have a dictionary named "images_dict" with image paths as keys and interests as values

# Check if the dictionary is not empty
if images_dict:
    # Get the first image path and interest
    first_image_path = list(images_dict.keys())[0]
    first_interest = images_dict[first_image_path]

    # Open and display the first image
    first_image = Image.open(first_image_path)
    first_image.show()
else:
    print("The dictionary is empty.")


In [3]:
import numpy as np
from PIL import Image

# Assuming you have a dictionary named "images_dict" with image paths as keys and interests as values
matching_interests = ['FOOD', 'TRAVEL', 'ARTS AND MUSIC', 'FASHION', 'FAMILY AND RELATIONSHIPS', 'BEAUTY', 'SPORTS', 'LIVE EVENTS']
# Check if the dictionary is not empty
if images_dict:
    # Initialize an empty NumPy array to store the image matrices
    image_array = np.empty((len(images_dict), 224, 224, 3), dtype=np.uint8)
    image_list=[]
    binary_lists=[]
    failed = 0
    # Iterate over the images in the dictionary
    for i, image_path in enumerate(images_dict.keys()):
        try:
            # Open the image
            image = Image.open(image_path)
            # Resize the image to (224, 224)
            image = image.resize((224, 224))
            # Convert the image to a NumPy array
            image_array = np.array(image)
            # Append the image matrix to the list
            image_list.append(image_array)
            binary_list = []
            for interest in matching_interests:
                if interest in images_dict[image_path]:
                    binary_list.append(1)  # Interest is present
                else:
                    binary_list.append(0)  # Interest is not present
            binary_lists.append(binary_list)
        except Exception as e:
            failed=failed+1
else:
    print("The dictionary is empty.")


In [4]:
print(len(image_list))

2120


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

In [6]:
input_shape = (224, 224, 3)
num_classes = 8
batch_size = 32
num_epochs = 5
# Model architecture
model = Sequential()
# Model architecture (VGG16 with boolean output of size 8)
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(8, activation='sigmoid')) 



In [7]:
# Compilation du modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(image_list, binary_lists, test_size=0.1, random_state=42)
y_train = np.array(y_train, dtype=int)
X_train = np.array(X_train)
y_test = np.array(y_test, dtype=int)
X_test = np.array(X_test)

In [9]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32)

Epoch 1/5
60/60 [==============================] - 728s 12s/step - loss: 20.2791 - accuracy: 0.3286 - val_loss: 0.3908 - val_accuracy: 0.3962
Epoch 2/5
60/60 [==============================] - 738s 12s/step - loss: 0.3997 - accuracy: 0.3559 - val_loss: 0.3920 - val_accuracy: 0.3962
Epoch 3/5
60/60 [==============================] - 727s 12s/step - loss: 0.3961 - accuracy: 0.3873 - val_loss: 0.3891 - val_accuracy: 0.3962
Epoch 4/5
60/60 [==============================] - 995s 17s/step - loss: 0.3967 - accuracy: 0.3648 - val_loss: 0.3906 - val_accuracy: 0.3962
Epoch 5/5
60/60 [==============================] - 845s 14s/step - loss: 0.3974 - accuracy: 0.3873 - val_loss: 0.3911 - val_accuracy: 0.3962


In [240]:
import numpy as np
from tensorflow.keras.preprocessing import image
img = image.load_img('C:\\Users\\Ahmed\\Desktop\\dataset\\4394849\\train\\BJrwvPfD1sD.jpg', target_size=(input_shape[0], input_shape[1], 3))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img / 255.0  # Normalize the image

In [241]:
pred_prob = model.predict(img)

1/1 [==============================] - 0s 62ms/step


In [242]:
print(pred_prob)

[[0.4659661  0.4601889  0.41826862 0.43916765 0.35718796 0.44696003
  0.48140022 0.48383355]]
